In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,  Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
import yfinance as yf
from pandas_datareader.data import DataReader
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import quandl
from fredapi import Fred
import statsmodels.api as sm



pd.set_option("display.max_rows",200)
sns.set(rc={'figure.figsize':(16,10)})
fred_key = "df4910b2cad947d95cf6ab16ba11d74d"
fred = Fred(api_key = fred_key)
quandl.ApiConfig.api_key = 'Qq5R29Xiqp2yUbb9dzNq'


In [2]:
def plot(prediction,target,grey=None,start=None,end=None):
    
    fig, ax = plt.subplots(figsize=(10, 3), dpi=300)
    
    # else:
    #     ylim = (df.min(),df.max())
    

    if not isinstance(prediction,pd.DataFrame):
        Results=pd.DataFrame(prediction,index=target.index)
        Results.plot(ax=ax,legend=True)
        ylim = (Results.min().min(),Results.max().max())
    else:
        prediction.plot(ax=ax,legend=True)
        ylim = (0,1)
    ax.fill_between(target.index, 0, ylim[1]+1e-2, target,facecolor='k', alpha=0.1)
    if not isinstance(prediction,pd.DataFrame):
        legend_list=["Prediction", "NBER recession indicator"]
    else:
        legend_list=["Prediction",'lower','upper', "NBER recession indicator"]
    # if '2022-02-24' in target.index:
    #     legend_list+=['Russian invasion of Ukraine']
    #     ax.axvline(x='2022-02-24', color='r')
    ax.legend(legend_list)
        
    return fig,ax

In [3]:
MacroCode=pd.read_csv(r'Data\Macro Variables.csv')
MacroCode.replace({'Average HOUST':'HOUST','S&P 500':'SP500'},inplace=True)

errors=[]
indicators={}
freq={}
for code in MacroCode['Variable']:
    if not ('S&P' in code):
        try:
            col=fred.get_series(code).to_frame(code).squeeze()
            if code == 'CPFF':
                indicators[code]=(col) #.resample('MS').last()
            elif code == "ICSA":
                indicators[code]=(col) #.resample('MS').sum()
            elif code == "SP500":
                indicators[code]=(col) #.resample('MS').first()
            else:
                indicators[code]=(col) # .resample('M').interpolate()
        except ValueError:
            errors.append(code)
indicators['S&P: indust']=(yf.download('^SP500-20')['Close'].to_frame('S&P: indust').squeeze()) # .resample("MS").last()
indicators['S&P div yield']=(quandl.get("MULTPL/SP500_DIV_YIELD_MONTH").squeeze().to_frame('S&P div yield').squeeze()) # .resample("MS").last()
indicators['S&P PE ratio']=(quandl.get("MULTPL/SP500_PE_RATIO_MONTH").squeeze().to_frame('S&P PE ratio').squeeze()) # .resample("MS").first()


[*********************100%***********************]  1 of 1 completed


In [4]:
if len(errors)>0:
    raise Exception("Erorrs found")
data=pd.concat(indicators,axis=1).copy() #.fillna(method='ffill') #.loc["1971":].dropna(how='all').fillna(method='ffill')
# data.to_csv('Data/HistoricalVariables.csv')
# print(data.shape)
# data.isna().sum().sort_values()/data.shape[0]

In [5]:
data=data.resample('D').last().fillna(method='ffill')

In [6]:
def deltaX(data):
    return data.diff()

def delta2X(data):
    return data-2*data.shift(1)+data.shift(2)

def logdata(data):
    return np.log(data)

def difflog(data):
    return np.log(data).diff()

def difflog2(data):
    return np.log(data)-2*np.log(data.shift(1))+np.log(data.shift(2))

def diffpercent(data):
    return (data/data.shift() - 1) - (data.shift()/data.shift(2) - 1)



transformation={1:(lambda x: x),
                2:deltaX,
                3:delta2X,
                4:logdata,
                5:difflog,
                6:difflog2,
                7:diffpercent,
                }

df_transformed=pd.DataFrame()

for column in data.columns:
    type=MacroCode[MacroCode['Variable'] == column]['Transformation'].iloc[0]
    df_transformed[column]=transformation[type](data[column])
    df_transformed=df_transformed.copy()

In [7]:
DD=data.interpolate().dropna(thresh=110).copy()

In [8]:
recessions =  DataReader('USRECDM', 'fred', start='1800')
DD['recessions']=recessions
train=DD.loc[:'2003']
test=DD.loc['2004':]

In [9]:

# Separate the independent variables (X) and the dependent variable (y)
X = train.drop(['recessions'],axis=1).fillna(0)  # Drop the target variable from the independent variables
y = train['recessions'].fillna(0)  # Target variable

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create and fit the model
model = sm.OLS(y, X)
results = model.fit()

# Print the summary of the regression analysis
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             recessions   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     587.3
Date:                Thu, 01 Jun 2023   Prob (F-statistic):               0.00
Time:                        18:09:07   Log-Likelihood:                 7199.9
No. Observations:               12053   AIC:                        -1.417e+04
Df Residuals:                   11937   BIC:                        -1.331e+04
Df Model:                         115                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               1.2002      0.873      1.375      0.169      -0.511       2.911
RPI                -0.0011      0.000     -6.939      0.000      -0.001      -0.001
W875RX1             0.0015      0.000      8.643      0.000       0.001       0.002
DPCERA3M086SBEA    -0.0795      0.009     -8.731      0.000      -0.097      -0.062
CMRMTSPL         8.718e-07    3.8e-07      2.292      0.022    1.26e-07    1.62e-06
RSXFS            7.082e-06   1.48e-06      4.790      0.000    4.18e-06    9.98e-06
INDPRO              1.7336      0.124     13.991      0.000       1.491       1.976
IPFPNSS            -1.0577      0.069    -15.354      0.000      -1.193      -0.923
IPFINAL             0.1468      0.028      5.300      0.000       0.093       0.201
IPCONGD            -0.3335      0.036     -9.392      0.000      -0.403      -0.264
IPDCONGD            0.0887      0.012      7.355      0.000       0.065       0.112
IPNCONGD            0.2532      0.023     11.055      0.000       0.208       0.298
IPBUSEQ            -0.1105      0.010    -11.335      0.000      -0.130      -0.091
IPMAT              -0.8648      0.064    -13.563      0.000      -0.990      -0.740
IPDMAT              0.0158      0.015      1.038      0.299      -0.014       0.046
IPNMAT             -0.0276      0.004     -6.546      0.000      -0.036      -0.019
IPMANSICS           0.0517      0.037      1.387      0.165      -0.021       0.125
IPB51222S          -0.0086      0.001     -5.831      0.000      -0.012      -0.006
IPFUELS            -0.0081      0.001     -6.796      0.000      -0.010      -0.006
CUMFNS              0.1028      0.008     12.177      0.000       0.086       0.119
CLF16OV            -0.0002   4.13e-05     -4.267      0.000      -0.000   -9.52e-05
CE16OV              0.0001   4.32e-05      2.517      0.012    2.41e-05       0.000
UNRATE              0.3049      0.039      7.790      0.000       0.228       0.382
UEMPMEAN            0.0164      0.005      3.160      0.002       0.006       0.027
UEMPLT5          1.253e-05   2.68e-05      0.468      0.640   -3.99e-05     6.5e-05
UEMP5TO14       -2.507e-05   2.92e-05     -0.859      0.390   -8.23e-05    3.21e-05
UEMP15OV           -0.0214      0.007     -3.057      0.002      -0.035      -0.008
UEMP15T26           0.0212      0.007      3.028      0.002       0.007       0.035
UEMP27OV            0.0214      0.007      3.056      0.002       0.008       0.035
ICSA             7.904e-07   6.74e-08     11.734      0.000    6.58e-07    9.22e-07
PAYEMS              0.0041      0.000     10.544      0.000       0.003       0.005
USGOOD              0.0081      0.001     10.511      0.000       0.007       0.010
CES1021000001      -0.0108      0.001     -9.294      0.000      -0.013      -0.009
USCONS             -0.0125      0.001    -10.769      0.000